### Importar bibliotecas

In [1]:
import numpy as np
import pandas as pd
import re as regex
import math as math

### Carga del dataset desde el archivo

In [3]:
pathArchivoDataSet = 'properatti.csv'
ds = pd.read_csv(pathArchivoDataSet)

In [4]:
ds.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

## Completar columna price_usd_per_m2

In [5]:
ds['price_usd_per_m2'].isna().sum()

52603

In [10]:
ds['price_per_m2'].isna().sum()

33562

In [6]:
def updatePriceUSDPerM2(dataFrameRow, dolarConversion):
    
    conversion = 1
    
    #si el dato currency no tiene valor, asumimos que los valores son en USD (conversion = 1)
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_usd_per_m2)):
        #precio por m2 USD= precio total USD / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
        
        #precio por m2 USD = (precio total $ * conversion moneda) / superficie total        
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
    
        #precio por m2 USD = precio por m2 * conversion moneda
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_per_m2)):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_per_m2 * conversion
            
        #precio por m2 USD = precio aprox por m2 USD / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
            
        #precio por m2 USD = precio aprox por m2 local currency * conversion moenda / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price_aprox_local_currency * conversion) / dataFrameRow.surface_total_in_m2
            
    return dataFrameRow

In [7]:
def updatePricePerM2(dataFrameRow, dolarConversion):
            
    #si el dato currency no tiene valor, asumimos que el valor precio por m2 a guardar es en ARS
    conversion = 1 / dolarConversion['ARS'] #si el dolar esta a 65 ARS => el factor de conversión de USD a ARS es 1 / 65
    
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = 1 / dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_per_m2)):
        #precio por m2 = precio aprox total local currency / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_local_currency / dataFrameRow.surface_total_in_m2
        
        #precio por m2 = precio por m2 USD / conversion moneda
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_usd_per_m2)):
            dataFrameRow.price_per_m2 = dataFrameRow.price_usd_per_m2 / conversion
            
        #precio por m2 USD = (precio aprox por m2 USD / conversion) / superficie total
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_per_m2 = (dataFrameRow.price_aprox_usd / conversion) / dataFrameRow.surface_total_in_m2
             
    return dataFrameRow

### Ejectuamos la actualización y luego verificamos la cantidad de rows que siguen sin tener dato en la columna price_usd_per_m2

In [11]:
conversion_USD_a_monedas_locales = { 'ARS': 63, 'PEN': 3.53, 'UYU': 43.41, 'USD': 1}

In [12]:
ds = ds.apply(updatePriceUSDPerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [13]:
ds['price_usd_per_m2'].isna().sum()

28295

### Ejectuamos la actualización y luego verificamos la cantidad de rows que siguen sin tener dato en la columna price_per_m2

In [14]:
ds = ds.apply(updatePricePerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [15]:
ds['price_per_m2'].isna().sum()

28295

In [ ]:
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 1000
ds[ds['price_usd_per_m2'].isna()].head(50)

### Obtener un grupo particular de un agrupamiento, a partir de una KEY

In [ ]:
def getGroupFromKey(grouped, keyTupla):
    try:
        #busca entre todos los grupos el que pertenece a la key (la key es de tipo Tupla)
        return grouped.get_group(keyTupla)
    except KeyError:
        #si no lo encuentra tira una exception, la capturamos y retornamos un DataFrame vacío para indicar que no lo encontró
        return pd.DataFrame()

In [ ]:
getGroupFromKey(groups, ('Argentina', 'Bs.As. G.B.A. Zona Norte', ' Country Maschwitz Club', 'house', 'USD', 4.0)).mean().price

### Recorremos el Data frame fila por fila, consultando al agrupamiento la media del valor que buscamos. Para buscar en el agrupamiento se usa una tupla como Key

In [ ]:
def getPriceUSDPerM2Mean(dataFrameRow):
    group = getGroupFromKey(groups, (dataFrameRow.country_name, dataFrameRow.state_name, dataFrameRow.place_name, dataFrameRow.property_type, dataFrameRow.currency, dataFrameRow.rooms))
    
    if(len(group) > 0):
        #existe un grupo para la key buscada, se toma la media
        dataFrameRow.price_usd_per_m2 = group.mean().price_usd_per_m2
    
    return dataFrameRow  
    

In [ ]:
#ds = ds.apply(getPriceUSDPerM2Mean, axis=1);

### Con el sigiuente código se pueden ver las descripciones que no entraron en el procesamiento

In [ ]:
pd.options.display.max_colwidth = 2000  # set a value as your need
pd.options.display.max_rows = 999
print(ds[ds['rooms'].isna() == True].head(100).description)